In [1]:
import pandas as pd
import numpy as np
import math
import random

#Map interactive
import folium as f

#Map pour connection
import matplotlib.pyplot as plt
%matplotlib inline

#Calcul great circle
import pyproj

import networkx as nx

# To Load File
import bz2
import pickle

from branca.element import Figure

import mercury as mr # for widgets


In [2]:
# TEST APP
show_code = mr.Checkbox(label="Voir le code", value=False)
app = mr.App(title="Calculs d'itinéraires", 
            description="Besoin de plannifier vos prochain déplacements ? Venez découvrir vos itinéraires les plus courts !", 
            show_code=show_code.value,
            continuous_update=False,
            allow_download=False)

mercury.Checkbox

mercury.App

In [3]:
# Importation des données

# Les vols
with bz2.BZ2File('./All_DfVols.bz2', 'r') as file:
    df_vols, _, _ = pickle.load(file)

# Load all dictionnaries from file using pickle using bz2 compression
with bz2.BZ2File('AllData_In_Dict.bz2', 'r') as file:
    dict_compagnies, dict_aeroports, dict_pays = pickle.load(file)

In [4]:
# Fonction de distance
def coordDepuisAeroport(iata : str):
    '''
    Fonction : Recherche les coordonnées GPS d'un aéroport
    Retour : lat : float ,lon : float --> Latitude & longitude de l'aéroport
    '''
    lat = float(dict_aeroports[iata]['Lat'])
    lon = float(dict_aeroports[iata]['Lon'])
    return lat,lon

def DistGrandCercle(lat1 : float, lon1 : float, lat2 : float, lon2 : float):
    '''
    Fonction :  Calcul la longueur du grand cercle (= cercle tracé à la surface 
                d'une sphère qui a le même diamètre qu'elle) entre 2 coordonnées
    Retour : Distance en Km
    '''
    #Conversion en radian
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    #Delta des coordonnées
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    #Rayon terrestre (en km)
    r = 6367.0

    #Formule d'Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    return 2 * r * math.asin(math.sqrt(a))


def DistGrandCercleICAO(icao1 : str, icao2 : str):
    '''
    Fonction :  Calcul la longueur du grand cercle (= cercle tracé à la surface 
                d'une sphère qui a le même diamètre qu'elle) entre 2 aéroports
    Retour : Distance en Km
    '''
    #Récupération des coordonnées des aéroports
    latDep, lonDep = coordDepuisAeroport(icao1)
    latArr, lonArr = coordDepuisAeroport(icao2)

    #On calcule la distance les séparant
    return DistGrandCercle(latDep,lonDep,latArr,lonArr)

def Dist(x1,y1,x2,y2):
    '''
    Fonction : Calcul la distance euclidienne entre deux points
    Retour : Distance
    '''
    return math.sqrt((y2-y1)**2+(x2-x1)**2)

# Fonction de calcul de coordonnées
def vectGrandCercle(latDep : float, lonDep : float, latArr : float, lonArr : float):
    '''
    Fonction : Calcul un ensemble de coordonnées permettant de tracer un grand cercle terrestre
    Return : 2 listes de coordonnées de forme [[lat,lon],[lat,lon]]
    '''

    #On initialise le module de calcul
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(lonDep, latDep, lonArr, latArr)

    # On découpe le chemin entre 2 coordonnée avec des segments <= 100 km
    lonlats = g.npts(lonDep, latDep, lonArr, latArr,
                    1 + int(dist / 100000))

    #Mise en forme
    v1 = []
    v1.append([latDep,lonDep])
    v2 = []
    _lon = lonDep
    _lat = latDep
    horscadre = False

    # On parcourt les segments afin de créer un vecteur de coordonnées [lat,lon]
    # et non [lon,lat] comme retourne la fonction g.npts
    # La notion "horscadre" défini le dépassement d'un planisphère (à l'est ou à l'ouest)
    # La longitude passe de -180 à l'ouest à + 180 à l'est
    # On détecte ce dépassement avec un calcul de distance euclidienne Dist(_lat,_lon,lat,lon) < 4
    for lon, lat in lonlats:
        if(not(horscadre) and Dist(_lat,_lon,lat,lon) < 4):        
            v1.append([lat,lon])
        else:
            horscadre = True
            v2.append([lat,lon])
        _lon = lon
        _lat = lat
    
    if(horscadre):
        v2.append([latArr,lonArr])
    else:
        v1.append([latArr,lonArr])

    return v1,v2

In [5]:
# Filtre des dictionnaires
def filtreDictAeroports(colonne : str, valeur : str):
    '''
    Fonction : Filtre les clés du dictionnaire en fonction d'une valeur 
    présente dans les valeurs associées.
    Return : Liste de code ICAO des aéroports
    '''
    Listtmp = []
    for (key, value) in dict_aeroports.items():
        if value[colonne] == valeur:
            Listtmp.append(key)
        if valeur == '':
            Listtmp.append(value[colonne])
    return Listtmp

def filtreDictCompagnies(colonne : str, valeur : str):
    '''
    Fonction : Filtre les clés du dictionnaire en fonction d'une valeur 
    présente dans les valeurs associées.
    Return : Liste de code ICAO des compagnies
    '''
    Listtmp = []
    for (key, value) in dict_compagnies .items():
        if value[colonne] == valeur:
            Listtmp.append(key)
        if valeur == '':
            Listtmp.append(value[colonne])

    return Listtmp

In [6]:
dict_route = {}

g = df_vols.groupby(['Depart','Arrivee'])
taille_groupe = g.size().items()

for liaison, nbrVols in taille_groupe:
    #Si la liaison est déjà dans le dict, on incrémente le nbr de vols
    if(frozenset(liaison) in dict_route):
        dict_route[frozenset(liaison)]['Vols'] += nbrVols
    #Sinon, on crée une nouvelle liaison
    else:
        dict_route[frozenset(liaison)] = {'Dist' : DistGrandCercleICAO(liaison[0],liaison[1]), 'Vols' : nbrVols}

In [7]:
### -----------------
### FONCTION D'AFFICHAGE
### -----------------
def creation_lines(path : list, Line_group, c):
    '''
    fonction qui cree les lignes pour le folium map
    path : Chemin, 
    line_group : Group de ligne, 
    c : Couleur de ligne 
    '''
    for element in range(len(path)-1) :  
        start = path[element]
        end = path[element+1]
        latDep, lonDep = coordDepuisAeroport(start)
        latArr, lonArr = coordDepuisAeroport(end)

        #Calcul des chemins à afficher
        v2 = []
        v1,v2 = vectGrandCercle(latDep,lonDep,latArr,lonArr)
        
        col_ligne = c
        col_marker = c 
        
        #-------------------
        f.PolyLine(locations=v1,weight=3, color=col_ligne).add_to(Line_group)
        if(len(v2) != 0):
            f.PolyLine(locations=v2,weight=3, color=col_ligne).add_to(Line_group)

        f.CircleMarker(location = [latArr, lonArr],
                        radius = 1.6, 
                        color = col_marker, 
                        tooltip=dict_aeroports[end]['City'],
                        popup="<b>Nom : </b>" + dict_aeroports[end]['Name'] + """<br />
                                <b>Code :</b> """ + dict_aeroports[end]['Iata'] + """<br />
                                <b>Coordonnées :</b> (""" + str(dict_aeroports[end]['Lat']) + ";" + str(dict_aeroports[end]['Lon']) + """)<br />
                                <b>City : </b>""" + str(dict_aeroports[end]['City']) + """<br />
                                <b>Pays : </b>""" + dict_aeroports[end]['Country']).add_to(Line_group)


def map_correspondance(collection_path, b : bool):
    '''
    fonction qui gere le ajoute de chaque ligne pour tous les chemins qui sont dans le collection path 
    collection sous la forme de {0 : {'distance' : .. , 'path' : []}, 1 : {..} , etc}
    '''
    
    # 1 group de line 
    Line_group = f.FeatureGroup(name = "Tous")

    # generer les differents couleurs pour les differents chemins 
    longeur = 0 
    if isinstance(collection_path, dict) : longeur = len(collection_path.keys())
    if isinstance(collection_path, list) : longeur = len(collection_path)
    rand_colors = []
    for _ in range(longeur):
        rand_colors.extend( ["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])] ) 
    
    # ajoute le marker du debut 
    air_start = str
    if isinstance(collection_path, dict) : air_start = collection_path[0]['path'][0]
    if isinstance(collection_path, list) : air_start = collection_path[0]['path'][0]
    latStart, lonStart = coordDepuisAeroport(air_start)
    
    m = f.Map(location=[latStart, lonStart], zoom_start=2)
    
    f.CircleMarker(location = [latStart, lonStart],
                        radius = 1, 
                        color = '#000000', 
                        tooltip=dict_aeroports[air_start]['City'],
                        popup="<b>Nom : </b>" + dict_aeroports[air_start]['Name'] + """<br />
                                <b>Code :</b> """ + dict_aeroports[air_start]['Iata'] + """<br />
                                <b>Coordonnées :</b> (""" + str(dict_aeroports[air_start]['Lat']) + ";" + str(dict_aeroports[air_start]['Lon']) + """)<br />
                                <b>City : </b>""" + str(dict_aeroports[air_start]['City']) + """<br />
                                <b>Pays : </b>""" + dict_aeroports[air_start]['Country']).add_to(Line_group)
    
    
    # si on desinner des lines a partir d'une dictionnary 
    if isinstance(collection_path, dict) : 
        # iteration sur le dict pour ajouter chaque chemin 
        for key, contenu in collection_path.items() : 
            # une couleur par chemin 
            couleur = rand_colors.pop()
            # creation de poly lignes 
            creation_lines(contenu['path'], Line_group, couleur)
        
    # si on desinner des lines a partir d'une liste 
    if isinstance(collection_path, list):
        for contenu in collection_path : 
            # une couleur par chemin 
            couleur = rand_colors.pop()
            # creation de poly lignes 
            creation_lines(contenu['path'], Line_group, couleur)
        
    #Ajout les lines à la carte
    Line_group.add_to(m)
    
    fig = Figure()
    fig.add_child(m)
    #display(fig)
    return m

In [8]:
### -----------------
### CREATION DU GRAPHES
### -----------------

array_arrivee = []
array_depart = [] 
#c creation de deux arrays : arrivee et depart 
for key in dict_route:

    if len(key) == 2 :
        depart, arrive = key 
        
        array_arrivee.append(arrive)
        array_depart.append(depart)

# creation de deux lists unique avec tous les code de airports : arrivee et depart  
list_arrivee = np.unique(array_arrivee)
list_depart = np.unique(array_depart)
# remplacer le code par le nom en entier de airports 
list_arrivee = list(map(lambda x : dict_aeroports[x]['Name'], list_arrivee))
list_depart = list(map(lambda x : dict_aeroports[x]['Name'], list_depart))

list_arrivee = sorted(list_arrivee)
list_depart = sorted(list_depart)

In [9]:
# creation du graph avec network x 
graph_totale = nx.Graph()
list_airports_codes = []; 
# outils pour debuggae 
compteur = 0 
list_debug = [] 

for key in dict_route:
    # si le clé contient deux code's de airport 
    if len(key) == 2 :
        start, end = key 
        compteur = compteur + 1
        # si le start n'existe pas dans le liste_airport_codes 
        if list_airports_codes.count(start) == 0:
            list_airports_codes.append(start)

        if list_airports_codes.count(end) == 0:
            list_airports_codes.append(end)
            
        index_start = list_airports_codes.index(start); 
        index_end = list_airports_codes.index(end); 
        distance = dict_route[frozenset((start,end))]['Dist']
        
        # ajoute aux graphe 
        graph_totale.add_node(index_start)
        graph_totale.add_node(index_end)
        
        graph_totale.add_edge(index_start, index_end, weight = distance)
    else: list_debug.append(key)

nb_keys = len(dict_route.keys())
#print('Nombre de keys dans le dict_route : ', print(nb_keys))
#debug = nb_keys - compteur
#print('Nombre de keys qui ne sont pas ajoute : ', debug , ', voici le tableau avec le keys qui ne sont pas ajoutee')
#print(list_debug)

# Save to ajdlist
# nx.write_adjlist(graph_totale, "graph_totale.bz2")

In [10]:
def dijkstra(graphe : nx.Graph, start : int, end : int):
    '''
        fonction qui utilise l'algorithme de Dijkstra pour calcule le plus court
        chemin et calcule le distance
        retour \: dict{\'distance\' : ... , \'path\' : ...}
    '''
    try :
        tmp1 = nx.dijkstra_path(graphe, start,end)
        dist = nx.dijkstra_path_length(graphe, start, end)
        return {'distance' : dist, 'path' : tmp1}
    except nx.NetworkXNoPath:
        print('no path (dijkstra)')
        return -1 

In [11]:
def supprimerJusqauNombre(g_modif : nx.Graph, nombre : int, index1 , index2):
    '''
        fonction qui supprime (temporarement) des connections entre airports pour trouver plusieurs chemins entre deux airports 
        retourne une dictionnary de solutions 
    '''
    index_compteur = 0 
    collection_solutions = dict() 
    
    # premiere solution (solution le plus vite )
    solution = dijkstra(g_modif, index1, index2)
    if(solution == -1):
        print('No path (supprimerJusquaNombre)')
        return -1 
    else : 
        collection_solutions[index_compteur] = solution; 
        # memoire pour remettre apres tous les connections 
        memoire = []
        
        while(len(solution['path']) < nombre ):
            # supprimer les connections dans le solution d'avant 
            ###################################
            for i in range(len(solution['path'])-1) :
                w = g_modif.get_edge_data(solution['path'][i], solution['path'][i+1])['weight']
                memoire.append([solution['path'][i], solution['path'][i+1], w])
                
                # delete les paths le plus vite --> on trouve les autres paths plus vite 
                g_modif.remove_edge(solution['path'][i], solution['path'][i+1])
            
            ###################################
            # trouve la nouvelle solution 
            solution = dijkstra(g_modif, index1, index2)
            if(solution == -1):
                print('No path (supprimerJusquaNombre)')
                return -1 
            else : 
                collection_solutions[index_compteur] = solution; 
                # ajouter la solution au dict de resultat 
                collection_solutions[index_compteur] = solution
                # augementer le compteur ( qui fonctionne comme cle dans le dict)
                index_compteur = index_compteur + 1 

        # remettre le graph au graph originale 
        g_modif.add_weighted_edges_from(memoire)
    
    return collection_solutions    

In [12]:
def chemin_PlusCourt(index1, index2, airport1, airport2): 
    '''
        fonction calculant le chemin le plus court entre deux airports
        retourne la solution 
    '''
    if graph_totale.has_node(index1):
        if graph_totale.has_node(index2):
            
            ###################### plus courtes ###########################
            
            resultat = dijkstra(graph_totale, index1, index2)
            if(resultat == -1 ):
                print('no path (plus_courte)')
                #return -1
            else : 
                s = list(map(lambda x : list_airports_codes[x], resultat['path']))
                solution_plus_courte = dict(); 
                solution_plus_courte[0] = {'distance' : resultat['distance'], 'path' : s }
                #map_correspondance(solution_plus_courte, False)
                return solution_plus_courte
        else : print(airport1, ' n\'est pas present dans le graphe')
    else : print(airport2, ' n\'est pas present dans le graphe')
    return -1

In [13]:
def chemin_N_escales(index1, index2, airport1, airport2, nb_escale, nb_routes = 5):
    '''
        fonction qui faire appele a supprimerJusquaNombre et qui fait tout le traitement 
        (avec les map inclus)
        index1 : Index de airport1 dans list_airports_codes
        index2 : Index de airport2 dans list_airports_codes
        airport1 : Code ICAO de airport1
        airport2 : Code ICAO de airport2
        nb_escale : Nombre d'escales
        nb_routes : Nombre de routes à calculer
    '''    
    if graph_totale.has_node(index1):
        if graph_totale.has_node(index2):
            dict_chemins = dict() 

            # 1 escale = 2 arcs dans le graphes
            nb_arc = nb_escale + 1
            for path in nx.all_simple_paths(graph_totale, index1, index2, nb_arc):
                # map the index to airport code
                airports = [list_airports_codes[i] for i in path]
                dict_chemins[len(dict_chemins)] = {
                    'distance' : nx.path_weight(graph_totale, path, "weight"), 
                    'path' : airports
                }                
                #print(nx.path_weight(graph_totale, path, "weight"), airports)
            if(len(dict_chemins) == 0):
                print('Pas de route avec ', nb_escale, ' escales')
                return -1
            # Trie les routes par ordres croissants de distance
            sorted_paths = sorted(dict_chemins.values(), key=lambda x: x['distance'])
            return sorted_paths[0:nb_routes]        
                
        else : print(airport1, ' n\'est pas present dans le graphe')
    else : print(airport2, ' n\'est pas present dans le graphe')
    return -1
#chemin_N_escales(234, 113, 'LYS', 'LAX', 0)

In [14]:
def chemin_escale_fixe(index1, index2, index_Escale):
    '''
        fonction qui faire appele a supprimerJusquaNombre et qui fait tout le traitement 
        (avec les map inclus)
        index1 : Index de airport1 dans list_airports_codes
        index2 : Index de airport2 dans list_airports_codes
        index_Escale : Index de l'escale dans list_airports_codes
        nb_escale : Nombre d'escales
        nb_routes : Nombre de routes à calculer
    '''    

    dict_chemins = dict() 

    # Calcul du chemin le plus court entre l'aéroport de départ et l'escale
    path1 = nx.shortest_path(graph_totale, index1, index_Escale, weight="weight")
    # Calcul du chemin le plus court entre l'escale et l'aéroport d'arrivée
    path2 = nx.shortest_path(graph_totale, index_Escale, index2, weight="weight")

    # Conversion en nom d'aéroport
    airports1 = [list_airports_codes[i] for i in path1]
    airports2 = [list_airports_codes[i] for i in path2]
    airports = airports1 + airports2[1:]
    dict_chemins[0] = {
        'distance' : nx.path_weight(graph_totale, path1, "weight") + nx.path_weight(graph_totale, path2, "weight"), 
        'path' : airports
    }
    return dict_chemins

In [15]:
def verif_widget(list_nom):
    '''
        fonction gérant le changement d'un des widgets 
        nom1 : Nom de l'aéroport de départ
        nom2 : Nom de l'aéroport d'arrivée
        nombre : Nombre d'escale
    '''

    for airport in list_nom : 
        if airport == '' : 
            print('Veuillez choisir un aéroport')
            return -1
        else :
            code_ICAO = filtreDictAeroports('Name', airport)[0]
            if list_airports_codes.count(code_ICAO) == 0 : 
                print(airport, 'sans connection')
                return -1

In [16]:
def map_itineraire_court(nom1, nom2):
    '''
        fonction gérant le changement d'un des widgets 
        nom1 : Nom de l'aéroport de départ
        nom2 : Nom de l'aéroport d'arrivée
        nombre : Nombre d'escale
    '''
    if(verif_widget([nom1, nom2]) == -1):
        return -1
    
    else :
        airport1 = filtreDictAeroports('Name', nom1)[0]
        airport2 = filtreDictAeroports('Name', nom2)[0] 
        index1 = list_airports_codes.index(airport1)
        index2 = list_airports_codes.index(airport2)

    # Chemin plus court : 
    # Code de Anne 2022
    # Conçue à la main pour comprendre le fonctionnement de l'algorithme
    res_court = chemin_PlusCourt(index1, index2, airport1, airport2)
    if(res_court != -1) : 
        m_court = map_correspondance(res_court, False)
        display(m_court)
    else :
        print('Pas de route disponible')

In [17]:
def map_itineraire_escale(nom1, nom2, nb_escales, nb_routes = 5):
    '''
        fonction gérant le changement d'un des widgets 
        nom1 : Nom de l'aéroport de départ
        nom2 : Nom de l'aéroport d'arrivée
        nombre : Nombre d'escale
    '''
    if(verif_widget([nom1, nom2]) == -1):
            return -1
        
    else :
        airport1 = filtreDictAeroports('Name', nom1)[0]
        airport2 = filtreDictAeroports('Name', nom2)[0] 
        index1 = list_airports_codes.index(airport1)
        index2 = list_airports_codes.index(airport2)
    
    # Chemin avec n escales : 
    # Code de Loris 2022
    # Conçue avec les fonctions intégrées à NetworkX pour trouver les chemins
    res_long = chemin_N_escales(index1, index2, airport1, airport2, nb_escales, nb_routes)
    if(res_long != -1) : 
        m_long = map_correspondance(res_long, False)
        display(m_long)
    else : print('Veuillez choisir plus d\'escales')

In [18]:
def map_itineraire_escale_fixe(nom1, nom2, nomEscale):
    '''
        fonction gérant le changement d'un des widgets 
        nom1 : Nom de l'aéroport de départ
        nom2 : Nom de l'aéroport d'arrivée
        nomEscale : Nom de l'escale
    '''
    
    if(verif_widget([nom1, nom2, nomEscale]) == -1):
        return -1
    else :        
        airport1 = filtreDictAeroports('Name', nom1)[0]
        airport2 = filtreDictAeroports('Name', nom2)[0] 
        airportEscale = filtreDictAeroports('Name', nomEscale)[0] 
        index1 = list_airports_codes.index(airport1)
        index2 = list_airports_codes.index(airport2)
        indexEscale = list_airports_codes.index(airportEscale)
    
    # Chemin avec escale fixe :
    # Code de Loris 2022
    # Conçue avec les fonctions intégrées à NetworkX pour trouver les chemins
    res_escale_fixe = chemin_escale_fixe(index1, index2, indexEscale)
    if(res_escale_fixe != -1) : 
        m_escale_fixe = map_correspondance(res_escale_fixe, False)
        display(m_escale_fixe)
    else : print('Pas de route disponible')

In [19]:
S_Airport1 = mr.Select(label="Selectionner un aéroport *", value="Lyon Saint Exupery Airport", choices=filtreDictAeroports('Name', ''))
S_Airport2 = mr.Select(label="Selectionner un aéroport *", value="Alice Springs Airport", choices=filtreDictAeroports('Name', ''))

mercury.Select

mercury.Select

In [20]:
mr.Md(f"### Trajet le plus court (km) entre _{S_Airport1.value}_ et _{S_Airport2.value}_")

### Trajet le plus court (km) entre _Lyon Saint Exupery Airport_ et _Alice Springs Airport_

In [21]:
map_itineraire_court(S_Airport1.value, S_Airport2.value)

In [22]:
S_escale = mr.Numeric(value=3, min=0, max=5, label="Nombre d'escale", step=1)

mercury.Numeric

In [23]:
if(S_escale.value == 0):
    mr.Md(f"### Trajet le plus court (km) entre _{S_Airport1.value}_ et _{S_Airport2.value}_ sans escale")
elif(S_escale.value == 1):
    mr.Md(f"### Trajet le plus court (km) entre _{S_Airport1.value}_ et _{S_Airport2.value}_ avec **{int(S_escale.value)} escale**")
else:
    mr.Md(f"### Trajet le plus court (km) entre _{S_Airport1.value}_ et _{S_Airport2.value}_ avec **{int(S_escale.value)} escales**")

### Trajet le plus court (km) entre _Lyon Saint Exupery Airport_ et _Alice Springs Airport_ avec **3 escales**

In [24]:
map_itineraire_escale(S_Airport1.value, S_Airport2.value, S_escale.value, 5)

In [25]:
S_Airport3 = mr.Select(label="Selectionner un aéroport de correspondance *", value="Montreal Pierre Elliott Trudeau Airport", choices=filtreDictAeroports('Name', ''))

mercury.Select

In [26]:
mr.Md(f"### Trajet le plus court (km) entre _{S_Airport1.value}_ et _{S_Airport2.value}_ en passant par **{S_Airport3.value}**")

### Trajet le plus court (km) entre _Lyon Saint Exupery Airport_ et _Alice Springs Airport_ en passant par **Montreal Pierre Elliott Trudeau Airport**

In [27]:
map_itineraire_escale_fixe(S_Airport1.value, S_Airport2.value, S_Airport3.value)